In [190]:
# Importing necessary packages
import pandas as pd
import mysql.connector
import warnings
warnings.filterwarnings('ignore')

In [191]:
# Function for mysql connector
def mysql_connector():
    try:
        # Connecting to mysql database using mysql connector
        mydb = mysql.connector.connect(host='localhost',user="root",password="0007")
        # Creating a cursor for running sql queries
        cursor = mydb.cursor()

    except Exception as e:
        logger(e)
        print(e)

In [192]:
# Creating a function to log all the errors in the process
import logging as lg
def logger(log):
    lg.basicConfig(filename='table_create.log',level=lg.ERROR,format="%(asctime)s %(name)s %(levelname)s %(message)s")
    lg.error(log)

### Loading and cleaning data 

In [193]:
# Function to load, clean and split data
def load_and_clean():
    # Loading all the data to a dataframe
    data = pd.read_excel(r"C:\Users\mdnis\Desktop\NUVENTO\source_target_field_map.xlsx")
    
    # Splitting Source and Target fields to different dataframes
    df1 = data[['SourceTableName','SourceColumnName']]
    df2 = data[['TargetTableName','TargetColumnName']]
    
    # Creating a new field which is concatenation of the two fields
    df1['SourceColumnName_New'] = df1['SourceTableName'] +" "+ df1['SourceColumnName']
    df2['TargetColumnName_New'] = df2['TargetTableName'] +" "+ df2['TargetColumnName']
    
    # Droping the unnecessary field
    df1.drop('SourceColumnName',axis=1,inplace=True)
    df2.drop('TargetColumnName',axis=1,inplace=True)
    
    # Dropping the null value columns from df2
    df2 = df2.dropna()
    
load_and_clean()

### Working with the dataframe-1 and dataframe-2

In [194]:
# Function to create all the tables with corresponding fields
def table_creator(data_frame_name, new_col_name, new_table_name, db_name):
    
    # Making a list containing all the table names to be created
    table_names = data_frame_name[f'{new_table_name}'].unique().tolist()
    
    # Selecting the database to work with
    cursor.execute(f"USE {db_name};")
    
    col_names_lst = []
    for i in table_names:
        tblnm = i
        col_names_lst = []
        for j in data_frame_name[f'{new_col_name}']:
            if j.split(" ")[0] == str(tblnm):
                col_names_lst.append(j.split(" ")[1])
        col_qry = " VARCHAR(50),".join(col_names_lst)
        tbl_qry = f"CREATE TABLE {tblnm} ({col_qry} VARCHAR(50));"
        cursor.execute(f'{tbl_qry}')
        

In [195]:
try:
    mysql_connector()
    table_creator(df1,'SourceColumnName_New','SourceTableName','new_database')
except Exception as e:
    print("Your Error is : ",e)
    logger(e)
else:
    stat = "Successfully Completed"
    lg.debug(stat)
    print(stat)

Successfully Completed


In [196]:
try:
    mysql_connector()
    table_creator(df2,'TargetColumnName_New','TargetTableName','new_database1')
except Exception as e:
    print("Your Error is : ",e)
    logger(e)
else:
    stat = "Successfully Completed"
    lg.debug(stat)
    print(stat)
    

Successfully Completed


### To Insert the data to corresponding table

In [199]:
# Function to normalize data to work for the insertion part
def ins_clean_data():
    data = pd.read_excel(r"C:\Users\mdnis\Desktop\NUVENTO\source_target_field_map.xlsx")
    data['Final'] = data.SourceTableName + " " + data.SourceColumnName + " " + data.TargetTableName + " " + data.TargetColumnName
    data.drop(['SourceTableName','SourceColumnName','TargetTableName','TargetColumnName'],axis=1,inplace=True)
    data = data['Final'].dropna()

try:
    ins_clean_data()
except Exception as e:
    print(e)
    logger(e)

In [200]:
# Creating a function to capture the corresponding tablenames into a list
def tbllist_capture():
    tbllst_new = []
    for i in data:
        tbllst = []
        tblnm1 = i.split(" ")[0]
        tblnm2 = i.split(" ")[2]
        #print(tblnm1,tblnm2)
        if tblnm1 not in tbllst:
            tbllst.append(tblnm1)
        if tblnm2 not in tbllst:
            tbllst.append(tblnm2)
        else:
            continue
        if tbllst not in tbllst_new:
            tbllst_new.append(tbllst)
        else:
            continue

try:
    tbllist_capture()
except Exception as e:
    print(e)
    logger(e)
else:
    print("Process completed Successfuly")

Process completed Successfuly


In [201]:
# Creating a function to capture the corresponding column names into a list
def collist_capture():   
    collst_new = []
    for i in tbllst_new:
        collst = [[],[]]
        for j in data:
            if j.split(" ")[0] == i[0] and j.split(" ")[2] == i[1]:

                if j.split(" ")[1] not in collst[0]:
                    collst[0].append(j.split(" ")[1])

                if j.split(" ")[3] not in collst[1]:
                    collst[1].append(j.split(" ")[3])

                else:
                    continue
        if collst not in collst_new:
            collst_new.append(collst)
        else:
            continue

            
try:
    collist_capture()
except Exception as e:
    print(e)
    logger(e)
else:
    print("Process completed Successfuly")

Process completed Successfuly


In [210]:
# Function to migrate data from one table to the corresponding table
def transfer_data():
    for i in range(16):
        qry1 = f"INSERT INTO new_database1.{tbllst_new[i][1]} (" + (",".join(collst_new[i][1]))
        qry2 = ") SELECT " + ",".join(collst_new[i][0]) + f" FROM new_database.{tbllst_new[i][0]};"

        final_qry = qry1 + qry2
        cursor.execute(final_qry)

try:
    transfer_data()
except Exception as e:
    print(e)
    logger(e)
else:
    print("Process completed Successfuly")

Process completed Successfuly


In [204]:
# Task completed successfuly